In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import glob

import pandas as pd

In [3]:
big_data = pd.DataFrame()
files = glob.iglob('backtests/bybit/' + '**/backtest_result.txt', recursive=True)
i = 0
for filename in files:
    #print(f'parsing : {filename}')
    with open(filename, 'r') as file:
        data = file.read().split('\n')
        d = {'file': filename.replace('backtests/bybit','')}
        current_state = 'long'
        for l in data:
            l = l.split('|')
            if len(l) == 4:
                key = l[1].strip()
                value = l[2].strip()

                if key == 'Passivbot Version':
                    d['v'] = value
                if key == 'Symbol':
                    d['s'] = value

                if key == 'Long':
                    current_state = 'long'
                    d['long'] = bool(value) or False
                if key == 'Short':
                    current_state = 'short'
                    d['short'] = bool(value) or False

                if key == 'Total gain' and current_state == 'long':
                    d['tgl_%'] = float(value.replace('%', '').strip())
                if key == 'Average daily gain' and current_state == 'long':
                    d['adgl_%'] = float(value.replace('%', '').strip())
                if key == 'Closest bankruptcy' and current_state == 'long':
                    d['bnkrl_%'] = float(value.replace('%', '').strip())
                if key == 'Lowest equity/balance ratio' and current_state == 'long':
                    d['ebrl'] = float(value.replace('%', '').strip())

                if key == 'Total gain' and current_state == 'short':
                    d['tgs_%'] = float(value.replace('%', '').strip())
                if key == 'Average daily gain' and current_state == 'short':
                    d['adgs_%'] = float(value.replace('%', '').strip())
                if key == 'Closest bankruptcy' and current_state == 'short':
                    d['bnkrs_%'] = float(value.replace('%', '').strip())
                if key == 'Lowest equity/balance ratio' and current_state == 'short':
                    d['ebrs'] = float(value.replace('%', '').strip())

        #d['adgt_%'] = d['adgl_%']+d['adgs_%']
        big_data = big_data.append(d, ignore_index=True)

big_data

C:\Users\skhan\AppData\Local\Temp/ipykernel_1050156/3399278970.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp/ipykernel_1050156/3399278970.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp/ipykernel_1050156/3399278970.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp/ipykernel_1050156/3399278970.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ign

,file,v,s,long,tgl_%,adgl_%,bnkrl_%,ebrl,short,tgs_%,adgs_%,bnkrs_%,ebrs
0,\ADAUSDT\plots\2022-03-10T124500\backtest_resu...,5.5,ADAUSDT,True,-394.3,0.630,100.0,0.9645,True,-660.70000,0.783,100.000,0.95250
1,\ATOMUSDT\plots\2022-03-08T230945\backtest_res...,5.5,ATOMUSDT,True,-1475.0,0.609,100.0,0.9639,True,-1085.00000,0.552,100.000,0.92420
2,\AXSUSDT\plots\2022-03-08T135416\backtest_resu...,5.5,AXSUSDT,True,-3640.0,0.856,100.0,0.9581,True,-3513.00000,1.070,91.250,0.82850
3,\BNBUSDT\plots\2022-03-08T181744\backtest_resu...,5.5,BNBUSDT,True,-828.2,1.150,100.0,0.9671,True,-621.70000,0.884,100.000,0.96260
4,\DOTUSDT\plots\2022-03-09T153345\backtest_resu...,5.5,DOTUSDT,True,-746.3,0.674,100.0,0.9590,True,-970.50000,0.795,100.000,0.95470
5,\FTMUSDT\plots\2022-03-09T224354\backtest_resu...,5.5,FTMUSDT,True,-1208.0,0.550,100.0,0.9487,True,0.05677,0.419,11.910,0.11880
6,\LINKUSDT\plots\2022-03-10T063226\backtest_res...,5.5,LINKUSDT,True,-281.5,0.400,100.0,0.9597,True,-819.50000,0.634,100.000,0.95550
7,\MATICUSDT\plots\2022-03-09T154507\backtest_re...,5.5,MATICUSDT,True,-571.4,0.443,100.0,0.9622,True,-1299.00000,0.675,100.000,0.94910
8,\MATICUSDT\plots\2022-03-09T155533\backtest_re...,5.5,MATICUSDT,True,-571.4,0.443,100.0,0.9622,True,-1299.00000,0.675,100.000,0.94910
9,\NEARUSDT\plots\2022-03-10T082547\backtest_res...,5.5,NEARUSDT,True,-731.4,0.313,100.0,0.9591,True,-93.82000,-2.090,5.849,0.07212


Now filter the dataframe

In [4]:
allowed_symbols = ["ETHUSDT", "XRPUSDT", "1000SHIBUSDT", "DOGEUSDT", "ADAUSDT",
                   "BNBUSDT", "MATICUSDT", "DOTUSDT", "SANDUSDT", "FTMUSDT",
                   "LTCUSDT", "LINKUSDT", "MANAUSDT", "ETCUSDT", "EOSUSDT",
                   "ATOMUSDT", "FILUSDT", "ICPUSDT", "ALICEUSDT", "LRCUSDT",
                   "BCHUSDT", "SUSHIUSDT", "XTZUSDT", "THETAUSDT", "CRVUSDT",
                   "ALGOUSDT", "CHZUSDT", "TRXUSDT", "VETUSDT", "CHRUSDT",
                   "ONEUSDT", "OMGUSDT", "XLMUSDT", "RUNEUSDT", "ENJUSDT",
                   "CELRUSDT", "SXPUSDT", "1INCHUSDT", "SRMUSDT", "COTIUSDT",
                   "ZECUSDT", "QTUMUSDT", "GRTUSDT", "DENTUSDT", "SFPUSDT",
                   "STORJUSDT", "IOTAUSDT", "HOTUSDT", "NEOUSDT", "COMPUSDT",
                   "LINAUSDT", "KAVAUSDT", "BATUSDT", "WAVESUSDT", "IOSTUSDT",
                   "ALPHAUSDT", "RLCUSDT", "HBARUSDT", "DODOUSDT", "XMRUSDT",
                   "SNXUSDT", "ANKRUSDT", "DASHUSDT", "REEFUSDT", "ZENUSDT",
                   "CVCUSDT", "RENUSDT", "ICXUSDT", "RSRUSDT", "SKLUSDT",
                   "NKNUSDT", "BELUSDT", "ONTUSDT", "OGNUSDT", "BLZUSDT",
                   "MTLUSDT", "CTKUSDT", "RVNUSDT", "LITUSDT", "MKRUSDT",
                   "OCEANUSDT", "ZILUSDT", "UNFIUSDT", "BANDUSDT", "YFIIUSDT",
                   "TOMOUSDT", "TRBUSDT", "KNCUSDT", "STMXUSDT", "ZRXUSDT",
                   "SCUSDT", "AKROUSDT", "XEMUSDT", "FLMUSDT", "BALUSDT",
                   "BTSUSDT", "DGBUSDT", "DEFIUSDT"]

allowed_symbols = ["ADAUSDT"]
big_data['adgt_%'] = big_data['adgl_%'] + big_data['adgs_%']
df = big_data[(big_data.s.isin(allowed_symbols))
              & (big_data['long'] == True)
              & (big_data['short'] == True)
              & (big_data['adgt_%'] > 0.1)
              & (big_data['bnkrl_%'] > 95.0)
              & (big_data['bnkrs_%'] > 95.0)
              & (big_data['ebrs'] > .85)
              & (big_data['ebrl'] > .85)
              & (big_data['v'] == '5.5')

    # & (big_data['tg_%'] > 10)
    # & (big_data['ebr'] > .85)
              ]
#df['total_adg'] = big_data['adgl_%'] + big_data['adgs_%']
df

,file,v,s,long,tgl_%,adgl_%,bnkrl_%,ebrl,short,tgs_%,adgs_%,bnkrs_%,ebrs,adgt_%
0,\ADAUSDT\plots\2022-03-10T124500\backtest_resu...,5.5,ADAUSDT,True,-394.3,0.63,100.0,0.9645,True,-660.7,0.783,100.0,0.9525,1.413


In [5]:
df.drop_duplicates(subset=['s', 'tgl_%','tgs_%','adgl_%','adgs_%','adgt_%', 'bnkrl_%', 'bnkrs_%', 'ebrl', 'ebrs', 'long', 'short'], keep='first',
                   inplace=True)
dx = df[['file','s','adgt_%','adgl_%','adgs_%','bnkrl_%', 'bnkrs_%','ebrl', 'ebrs']]
dx

C:\Users\skhan\AppData\Local\Temp/ipykernel_1050156/2043545472.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(subset=['s', 'tgl_%','tgs_%','adgl_%','adgs_%','adgt_%', 'bnkrl_%', 'bnkrs_%', 'ebrl', 'ebrs', 'long', 'short'], keep='first',


,file,s,adgt_%,adgl_%,adgs_%,bnkrl_%,bnkrs_%,ebrl,ebrs
0,\ADAUSDT\plots\2022-03-10T124500\backtest_resu...,ADAUSDT,1.413,0.63,0.783,100.0,100.0,0.9645,0.9525
